# **KoBERT Model with fine tuning**

 - 2번분석, 발화 내용 기반 보이스피싱 발화문, 일반 상담 발화문 감정 판별

In [1]:
# # 원본 https://github.com/SKTBrain/KoBERT에 나와있는 using pytorch 코드
# !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [2]:
# # RUN
# 필요 import 문 
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np  # 'ersion : '1.23.5'
from tqdm.notebook import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

/anaconda/envs/mldl_torch/lib/python3.8/site-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


In [3]:
# # 원본 https://github.com/SKTBrain/KoBERT에 나와있는 using pytorch 코드
# from kobert import get_tokenizer
# from kobert import get_pytorch_kobert_model

In [4]:
# # 원본 https://github.com/SKTBrain/KoBERT에 나와있는 using pytorch 코드
# bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

### HuggingFace 우회

In [5]:
# # RUN
# ModuleNotFoundError: No module named 'kobert'로 인한 허깅페이스 우회
# !pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

In [6]:
# # RUN
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

In [7]:
# # RUN
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

In [8]:
# RUN
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

2023-08-13 12:48:28.230979: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-13 12:48:28.270647: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-13 12:48:28.272161: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-13 12:48:29.221483: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [9]:
# # RUN
# # 그래픽 카드 사용
print('torch version: {}'.format(torch.__version__))

if torch.cuda.is_available() == True:
    device = 'cuda:0'
    print('현재 가상환경 GPU 사용 가능상태')
else:
    device = 'cpu'
    print('GPU 사용 불가능 상태')

torch version: 1.10.1+cu102
현재 가상환경 GPU 사용 가능상태


In [10]:
# 사용하는 Graffic card 이름 확인
torch.cuda.is_available()
torch.cuda.get_device_name()

'NVIDIA GeForce RTX 2080'

In [11]:
# GPU 용량 확인
# Get the current memory usage (in bytes)
allocated = torch.cuda.memory_allocated(device)
reserved = torch.cuda.memory_reserved(device)

# Convert to MB
allocated_mb = allocated / 1024 / 1024
reserved_mb = reserved / 1024 / 1024

print(f'Allocated memory: {allocated_mb:.2f} MB')
print(f'Reserved memory: {reserved_mb:.2f} MB')

Allocated memory: 0.00 MB
Reserved memory: 0.00 MB


In [16]:
##### KoBERT 모델 관련 코드 #####
class BERTClassifier(nn.Module):
    def __init__(self, bert, hidden_size=768, num_classes=4, dr_rate=None, params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size, num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids=token_ids, token_type_ids=segment_ids.long(), attention_mask=attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [17]:
# model 정의

model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

### Model Load

In [18]:
# Load the best model
model_path = '/KITA_Project/Final Model for KoBERT/final_model_koBERT_vp_sem_clf.pt'  # adjust the path if needed
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [20]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, vocab, pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


## predict with real data

In [21]:
def predict(predict_sentence):
    data = [predict_sentence, '0']
    dataset_another = [data]
    
    # Define the maximum length for the tokens. You can adjust this value.
    max_len = 64
    
    another_test = BERTDataset(dataset_another, 0, 1, tokenizer, max_len, vocab, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=1, num_workers=5)
    
    model.eval()
    
    sentiment_labels = ["중립", "화남", "역겨움", "두려움"]

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)
        probs = F.softmax(out, dim=1)  # Convert logits to probabilities

        # Get the predicted label and its probability
        predicted_label_idx = torch.argmax(out).item()
        predicted_label_prob = probs[0][predicted_label_idx].item() * 100  # Convert to percentage

        print(f">> 입력하신 문장은 {predicted_label_prob:.2f}%의 확률로 \"{sentiment_labels[predicted_label_idx]}\"감정이 느껴지는 문장입니다.")

# User input loop
end = 1
while end == 1:
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0":
        break
    predict(sentence)
    print("\n")


하고싶은 말을 입력해주세요 : 안녕하세요
>> 입력하신 문장은 99.51%의 확률로 "중립"감정이 느껴지는 문장입니다.


하고싶은 말을 입력해주세요 : 네, 쿠폰은 같이 드립니다. 단 삼 단계는 현재 재고가 다 소진되어 있습니다.
>> 입력하신 문장은 96.14%의 확률로 "화남"감정이 느껴지는 문장입니다.


하고싶은 말을 입력해주세요 : 이용권에 따라 다릅니다.
>> 입력하신 문장은 90.92%의 확률로 "화남"감정이 느껴지는 문장입니다.


하고싶은 말을 입력해주세요 : 그러면 지금 스마트 뱅킹이나 인터넷 뱅킹을 사용하고 계신 건가요?
>> 입력하신 문장은 92.77%의 확률로 "화남"감정이 느껴지는 문장입니다.


하고싶은 말을 입력해주세요 : 저희 농협 대출 나가면 그거 가지고 친구분한테 빌렸던 거 갚아주면 되는 거니까요.
>> 입력하신 문장은 98.68%의 확률로 "화남"감정이 느껴지는 문장입니다.


하고싶은 말을 입력해주세요 : 0
